In [ ]:
%%capture
!pip install torch torchvision opencv-python opencv-python-headless matplotlib yolov5
!pip install grad-cam
!pip install rembg
!pip install onnxruntime

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%%capture

!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/narendraakumar/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Access the version using detectron2.__metadata__.__version__
try:
  print("detectron2:", detectron2.__metadata__.__version__)
except AttributeError:
  print("detectron2 version information not found. Please check your installation.")

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.5 ; cuda:  cu121
detectron2 version information not found. Please check your installation.


In [ ]:
%%capture
# clone and install Mask2Former
!git clone https://github.com/narendraakumar/Mask2Former.git
%cd Mask2Former
# !pip install -U opencv-python
!pip install git+https://github.com/cocodataset/panopticapi.git
!pip install -r requirements.txt
%cd mask2former/modeling/pixel_decoder/ops
!python setup.py build install
%cd ../../../../

In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
%cd /content/Mask2Former
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
setup_logger(name="mask2former")

# import some common libraries
import numpy as np
import cv2
import torch
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from detectron2.projects.deeplab import add_deeplab_config
coco_metadata = MetadataCatalog.get("coco_2017_val_panoptic")

# import Mask2Former project
from mask2former import add_maskformer2_config

/content/Mask2Former


/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/content/Mask2Former/mask2former/modeling/pixel_decoder/msdeformattn.py:314: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)


In [ ]:

cfg = get_cfg()
add_deeplab_config(cfg)
add_maskformer2_config(cfg)
cfg.merge_from_file("configs/coco/panoptic-segmentation/swin/maskformer2_swin_large_IN21k_384_bs16_100ep.yaml")

# Set the model weights
cfg.MODEL.WEIGHTS = 'https://dl.fbaipublicfiles.com/maskformer/mask2former/coco/panoptic/maskformer2_swin_large_IN21k_384_bs16_100ep/model_final_f07440.pkl'

# Enable different outputs
cfg.MODEL.MASK_FORMER.TEST.SEMANTIC_ON = True
cfg.MODEL.MASK_FORMER.TEST.INSTANCE_ON = True
cfg.MODEL.MASK_FORMER.TEST.PANOPTIC_ON = True

# Dynamically set device based on availability
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize the predictor
predictor = DefaultPredictor(cfg)

print(f"Using device: {cfg.MODEL.DEVICE}")

/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/17 05:19:49 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/maskformer/mask2former/coco/panoptic/maskformer2_swin_large_IN21k_384_bs16_100ep/model_final_f07440.pkl ...


model_final_f07440.pkl: 866MB [00:16, 54.0MB/s]                           


WARNING [12/17 05:20:09 mask2former.modeling.transformer_decoder.mask2former_transformer_decoder]: Weight format of MultiScaleMaskedTransformerDecoder have changed! Please upgrade your models. Applying automatic conversion now ...
Using device: cpu


In [ ]:
import os
import zipfile

def unzip_file(zip_file_path, extract_to_path="."):
  """Unzips a zip file to the specified directory.

  Args:
      zip_file_path: The path to the zip file.
      extract_to_path: The directory to extract the files to (defaults to current directory).
  """

  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

  print(f"File '{zip_file_path}' unzipped to '{extract_to_path}'")

# Example usage:
zip_file_path = "/content/Archive11.zip" # Replace with your zip file path
extract_to_path = "/content/test5" # Replace with your desired extraction directory
# unzip_file(zip_file_path, extract_to_path)

In [ ]:

import os,shutil
from PIL import Image

def resize_image(im_path,w=800,h=800, output_base_dir="/content/tmp/"):
    """
    Resize an image if its width or height exceeds 1000 pixels and save to a new directory
    while keeping the folder structure the same.

    Args:
        im_path (str): Path to the original image.
        output_base_dir (str): Base directory for resized images.

    Returns:
        str: Path to the resized image or original image.
    """
    # Open the original image
    # if os.path.exists():
    # shutil.rmtree(output_base_dir)
    image = Image.open(im_path)

    # Check if resizing is needed
    if image.width > w or image.height > h:
        # Calculate new dimensions
        new_width = int(w)
        new_height = int(h)

        # Resize using high-quality interpolation
        resized_image = image.resize((new_width, new_height), Image.LANCZOS)

        # Ensure the output directory structure is preserved
        parent_dir = os.path.join(im_path, os.pardir+"/"+os.pardir)
        relative_path = os.path.relpath(im_path, start=parent_dir)

        new_path = os.path.join(output_base_dir, relative_path)

        # Add file extension if it's missing
        if not os.path.splitext(new_path)[1]:
            new_path += os.path.splitext(im_path)[1]

        os.makedirs(os.path.dirname(new_path), exist_ok=True)

        # Save the resized image
        resized_image.save(new_path)
        print(f"Image resized and saved: {new_path}")
        return new_path
    else:
        print("Image dimensions are below 1000px; no resizing performed.")
        return im_path


In [ ]:
%matplotlib inline
import torch
import cv2
import numpy as np
from matplotlib import pyplot as plt
from rembg import remove
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import preprocess_image
from torchvision.models import resnet50
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from skimage.filters import sobel
from skimage import img_as_float, color
from PIL import Image

# Load ResNet50 model for Grad-CAM
resnet_model = resnet50(pretrained=True)
grad_cam = GradCAM(model=resnet_model, target_layers=[resnet_model.layer4[-1]])



def remove_background(image_path):
    input_image = Image.open(image_path)
    bg_removed = remove(input_image)  # Remove background using rembg
    bg_removed_np = np.array(bg_removed)  # Convert to numpy array for OpenCV processing
    return bg_removed_np

def remove_background1(image_path):
    # Load the original image
    input_image = Image.open(image_path).convert("RGBA")
    # Use rembg to remove background
    bg_removed = remove(input_image)

    # Convert both images to numpy arrays
    bg_removed_np = np.array(bg_removed)  # Hero product with transparent background
    original_np = np.array(input_image)    # Original image

    # Create a mask from the alpha channel of bg_removed_np (non-zero where hero product is)
    mask = bg_removed_np[:, :, 3] > 0  # Alpha channel

    # Use the mask to isolate the background from the original image
    background_only = original_np.copy()
    background_only[mask] = [0, 0, 0, 0]  # Make the hero product area transparent (white background)
    original_np[mask] = [0,0,0,0]
    # rgb_image = original_np.convert("RGB")
    rgb_image = original_np[:, :, :3]


    return bg_removed_np, background_only, rgb_image


def calculate_prominence(bg_removed_np, bbox, threshold=0.4):
    # Create a binary mask of the hero product from the RGBA image
    hero_product_mask = cv2.cvtColor(bg_removed_np, cv2.COLOR_RGBA2GRAY)
    hero_product_mask = (hero_product_mask > 0).astype(np.uint8)

    # Calculate hero product area from bounding box dimensions
    hero_product_area = (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
    img_area = hero_product_mask.shape[0] * hero_product_mask.shape[1]
    prominence_ratio = hero_product_area / img_area

    # Calculate initial prominence score based on area ratio
    prominence_score = 1 + (prominence_ratio * 3)
    prominence_score = np.round(prominence_score)

    # Set threshold for prominence score adjustments
    hero_product_width = bbox[2] - bbox[0]
    hero_product_height = bbox[3] - bbox[1]
    img_width = hero_product_mask.shape[1]
    img_height = hero_product_mask.shape[0]

    # Adjust prominence score based on area and proportions
    # print("hero_product_area ,img_area",hero_product_area, img_area,hero_product_area/img_area)
    # print("hero_product_height ,img_height",hero_product_height, img_height, hero_product_height/img_height)

    if hero_product_area > img_area * threshold:
        prominence_score = 4
        # print("hero_product_area > img_area * threshold")
    elif hero_product_height > img_height * 0.45:
        # print("hero_product_height > img_height * 0.5")
        # If height is more than half the image height but width is narrower
        if hero_product_width > img_width * 0.2:
            # print("hero_product_width > img_width * 0.2")
            # Set prominence score to a higher end if width > 20%
            prominence_score = 3.2
        else:
            # print("else")
            # Set prominence score to a lower end if width <= 20%
            prominence_score = min(4,1 + ((hero_product_width / (img_width * 0.2)) * 3.2))
            prominence_score = np.round(prominence_score, 2)
    elif hero_product_width > img_width * 0.4 and hero_product_height > img_height * 0.3:
        # print("hero_product_width > img_width * 0.4 and hero_product_height > img_height * 0.3")
        # If width is more than 40% and height is more than 30%
        # If both width and height cover a reasonable area
        prominence_score = 4
    else:

        # Set prominence score to a lower end if width <= 40% and height <= 30%
        prominence_score = min(4, (prominence_score / 6) * 7)
        # print("else")
    print("np.round(prominence_score, 2)",np.round(prominence_score, 2))

    return np.round(prominence_score, 2)

# def is_centered_score2(hero_product_mask,im=None, bbox=None,threshold=0.1):
#     coords = cv2.findNonZero(hero_product_mask)
#     # x, y, w, h = cv2.boundingRect(coords)
#     x, y, w, h = bbox[0],bbox[1],bbox[2]-bbox[0],bbox[3]-bbox[1]
#     # print("hero product bounding box",x,y,w,h)
#     img_center = (int(hero_product_mask.shape[1] / 2), int(hero_product_mask.shape[0] / 2))
#     product_center = (int((x + x+ w) / 2), int((y +y+ h )/ 2))
#     # print("hero_product_mask shape",hero_product_mask.shape,"image shape",im.shape)
#     # print("image center",img_center,"product center",product_center)
#     point_radius = 6  # Small radius for the point
#     point_color = (0, 0, 255)  # Red color in BGR format
#     cv2.circle(im, img_center, point_radius, point_color, thickness=4)
#     point_color = (255, 0, 0)
#     cv2.circle(im, product_center, point_radius, point_color, thickness=4)
#     cv2.rectangle(im, (x, y), (x+w, y+h), (0, 255, 0), 3)
#     cv2_imshow(im)
#     offset = np.linalg.norm(np.array(img_center) - np.array(product_center))
#     max_offset = np.linalg.norm(np.array(img_center)- np.array([int(w/2),int(h/2)]))

#     # Calculate the score based on offset, mapping to a 1-10 scale
#     # print("max offset:",max_offset,"offset",offset)
#     if offset <= max_offset*threshold:
#         # print(offset/max_offset)
#         score = 3.9
#     else:
#         # offset_ratio = min(offset / max_offset, 2)  # Cap the ratio to prevent excessive drop
#         # score = max(1, 10 - (offset_ratio - 1) * 9)  # Linear drop after threshold
#         score = ((1-offset/max_offset))*4

#     return np.round(score,2)
def is_centered_score(hero_product_mask, im=None, bbox=None, threshold=0.1):
    coords = cv2.findNonZero(hero_product_mask)
    x, y, w, h = bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]

    img_center = (int(hero_product_mask.shape[1] / 2), int(hero_product_mask.shape[0] / 2))
    product_center = (int((x + x + w) / 2), int((y + y + h) / 2))

    # Visualization (Optional)
    if im is not None:
        point_radius = 6  # Small radius for the point
        cv2.circle(im, img_center, point_radius, (0, 0, 255), thickness=4)  # Image center
        cv2.circle(im, product_center, point_radius, (255, 0, 0), thickness=4)  # Product center
        cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 3)  # Bounding box
        cv2_imshow(im)

    # Calculate offsets for x and y axes
    x_offset = abs(img_center[0] - product_center[0])  # Horizontal offset
    y_offset = abs(img_center[1] - product_center[1])  # Vertical offset

    # Maximum allowable offsets
    max_x_offset = hero_product_mask.shape[1] / 2  # Width / 2
    max_y_offset = hero_product_mask.shape[0] / 2  # Height / 2

    # Calculate individual scores
    x_score = 1 - (x_offset / max_x_offset)
    y_score = 1 - (y_offset / max_y_offset)

    # Weighted score
    total_score = (0.65 * x_score) + (0.35 * y_score)

    # Map to a 1-4 scale based on threshold
    if total_score >= 1 - threshold:
        final_score = 3.9
    else:
        final_score = total_score * 4

    return np.round(final_score, 2)

# Main processing and grading function
def centering_prominance(image_path,bbox):
    image = cv2.imread(image_path)

    # Background removal to extract hero product
    bg_removed_np, background_only,ip = remove_background1(image_path)
    # Focus Check - Grad-CAM
    # plt.imshow(cv2.cvtColor(bg_removed_np, cv2.COLOR_BGR2RGB))
    # plt.title("bg removed np")
    # plt.show()


    # plt.imshow(cv2.cvtColor(ip, cv2.COLOR_BGRA2RGBA))
    # plt.title("Remaining Background Only")
    # plt.show()

    # Calculate prominence
    prominence_ratio = calculate_prominence(bg_removed_np,bbox)
    hero_product_mask = cv2.cvtColor(bg_removed_np, cv2.COLOR_RGBA2GRAY)
    hero_product_mask = (hero_product_mask > 0).astype(np.uint8)
    # centered,bbox = is_centered(hero_product_mask)
    centered_score = is_centered_score(hero_product_mask,image,bbox)
    # print("Centered:", centered)
    print("Centered score:", centered_score)

    # Focus Check - Grad-CAM
    # import torchvision.models as models
    # import torch

    # Load a pre-trained ResNet model
    # model = models.resnet50(pretrained=True)
    # model.eval()
    # gradcam_result,grayscale_heatmap = generate_gradcam_heatmap(image,model)
    # plt.imshow(cv2.cvtColor(gradcam_result, cv2.COLOR_BGR2RGB))
    # plt.title("Grad-CAM Focus Map")
    # plt.show()

    # Saliency Check
    # saliency_map = generate_saliency_map(image)
    # plt.imshow(saliency_map, cmap='gray')
    # plt.title("Saliency Map")
    # plt.show()

    # Decide if focus is on hero product by inspecting saliency overlaps (for demo assume True)
    # focus = True
    # focus, focus_ratio = calculate_focus(bbox, grayscale_heatmap, saliency_map)
    # print("Focus on hero product:", focus, "with overlap ratio:", focus_ratio)

    return centered_score, prominence_ratio


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 140MB/s]


In [ ]:
import torch
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import preprocess_image
from torchvision.models import resnet50
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from skimage.filters import sobel
from skimage import img_as_float, color
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

resnet_model = resnet50(pretrained=True)
grad_cam = GradCAM(model=resnet_model, target_layers=[resnet_model.layer4[-1]])



def detect_hero_product(image_path):
    image = cv2.imread(image_path)
    results = yolo_model(image)
    detections = results.pandas().xyxy[0]
    print("#######")
    print(detections['name'])
    hero_product = detections
    # import pdb; pdb.set_trace()
    if hero_product.empty:
        print("Hero product not detected.")
        return None
    else:
      for _, product in hero_product.iterrows():
        top_hero = product
        bbox = (int(top_hero['xmin']), int(top_hero['ymin']), int(top_hero['xmax']), int(top_hero['ymax']))
        cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
    return bbox,image
# image_path = '/content/tes2/7/1.png'
# detect_and_draw_bboxes(image_path)
# bbox,img = detect_hero_product(image_path)
# print(bbox)
# cv2_imshow(img)

/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2024-12-17 Python-3.10.12 torch-2.5.1+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 115MB/s] 

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import os
import numpy as np
import cv2  # For image manipulation
from google.colab.patches import cv2_imshow
from PIL import Image
import os
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

def remove_transparancy(image_path):

  image = Image.open(image_path)

  if image.mode in ('RGBA', 'LA') or (image.mode == 'P' and 'transparency' in image.info):
      white_bg = Image.new("RGB", image.size, (255, 255, 255))
      white_bg.paste(image, mask=image.split()[3])
      image = white_bg
      image.save(image_path)



model = YOLO("yolov8x.pt")

def detect_and_draw_bboxes(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found at path: {image_path}")

    results = model(image)
    bbox = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            if abs(x2-x1)<100 and abs(y2-y1)<100:
                continue
            cls = int(box.cls.item())  # Class ID as integer
            conf = float(box.conf.item())  # Confidence score as float
            label = f"{cls} ({conf:.2f})"  # Class label with confidence score

            # Draw the bounding box and label
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            bbox.append([x1,y1,x2,y2])
    return bbox,image

def get_objects(img_path):
  remove_transparancy(img_path)
  im = cv2.imread(img_path)
  # cv2_imshow(im)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1], coco_metadata, scale=1.2, instance_mode=ColorMode.IMAGE_BW)
  panoptic_result = v.draw_panoptic_seg(outputs["panoptic_seg"][0].to("cpu"), outputs["panoptic_seg"][1]).get_image()
  cv2_imshow(panoptic_result)

  panoptic_seg, segments_info = outputs["panoptic_seg"]
  panoptic_seg_np = panoptic_seg.to("cpu").numpy()

  im_with_boxes = im.copy()
  bbox = []
  for segment in segments_info:
    segment_id = segment['id']  # Unique ID for this segment
    mask = panoptic_seg_np == segment_id  # Boolean mask for this segment

    # Find coordinates of non-zero (True) pixels
    # coords = np.argwhere(mask)
    # if coords.size > 0:
    #     # Compute bounding box (x_min, y_min, x_max, y_max)
    #     y_min, x_min = coords.min(axis=0)
    #     y_max, x_max = coords.max(axis=0)
    #     bbox.append([x_min, y_min, x_max, y_max])
  for segment in segments_info:
      segment_id = segment["id"]
      mask = (panoptic_seg_np == segment_id).astype(np.uint8)
      contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      for contour in contours:
          x, y, w, h = cv2.boundingRect(contour)
          cv2.rectangle(im_with_boxes, (x, y), (x + w, y + h), (0, 255, 0), 2)
          bbox.append([x,y,x+w,y+h])

  return bbox,im_with_boxes

def add_bbox_to_image(image_path, bbox, color=(0, 255, 0), thickness=2):
    image = cv2.imread(image_path)
    for x, y, x_max, y_max in bbox:
        cv2.rectangle(image, (x, y), (x_max, y_max), color, thickness)
    return image


def get_clip_embedding(image):
    if image.mode != 'RGB':
      image = image.convert('RGB')
    inputs = clip_processor(images=image, return_tensors="pt")
    with torch.no_grad():
        embeddings = clip_model.get_image_features(**inputs)
    return embeddings / embeddings.norm(dim=-1, keepdim=True)  # Normalize

def crop_objects(image_path, regions):
    image = Image.open(image_path).convert("RGB")
    cropped_objects = []
    for region in regions:
        x_min, y_min, x_max, y_max = region
        cropped_object = image.crop((x_min, y_min, x_max, y_max))
        cropped_objects.append((cropped_object,(x_min, y_min, x_max, y_max)))
    return cropped_objects

def find_closest_object(target_image_path, target_regions, object_images_path, object_regions):
    target_objects = crop_objects(target_image_path, target_regions)
    # for i, obj in enumerate(target_objects):
    #   obj[0].save(f"/content/{i}.png")
    target_embeddings = [(get_clip_embedding(obj[0]),obj[1]) for obj in target_objects]

    closest_image_path = None

    candidate_objects = crop_objects(object_images_path, object_regions)
    # for i, obj in enumerate(candidate_objects):
    #   obj[0].save(f"/content/{100+i}.png")
    candidate_embeddings = [(get_clip_embedding(obj[0]),obj[1]) for obj in candidate_objects]

    bbox = []
    for candidate_embedding in candidate_embeddings:
        max_similarity = -1
        for target_embedding in target_embeddings:
            similarity = torch.cosine_similarity(target_embedding[0], candidate_embedding[0]).item()
            # print(similarity)
            if similarity > max_similarity:
                max_similarity = similarity
                # print(candidate_embedding[1],target_embedding[1])
                tar=target_embedding[1]
                bbox=[tar]
    print("bbox",bbox)
    return (min([b[0] for b in bbox]),min([b[1] for b in bbox]),max([b[2] for b in bbox]),max([b[3] for b in bbox]))

# def get_image_bbox(image_path,silo_path):
#     target_regions,im1 = get_objects(image_path)
#     if len(target_regions)==0:
#       target_regions,im2 = detect_and_draw_bboxes(image_path)
#     object_regions,im3 = get_objects(silo_path)
#     if len(object_regions)==0:
#       object_regions,im4 = detect_and_draw_bboxes(silo_path)
#       # cv2_imshow(im4)

#     bbox = find_closest_object(image_path, target_regions, silo_path, object_regions)
#     im = add_bbox_to_image(image_path, [bbox])
#     cv2_imshow(im)
#     # cv2_imshow(im1)
#     # cv2_imshow(im2)
#     return bbox
def get_image_bbox(image_path, silo_path):

    import os
    import pickle
    pickle_file = image_path.split(".")[0]+".pkl"
    # Load existing cache if the pickle file exists
    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as f:
            bbox_cache = pickle.load(f)
    else:
        bbox_cache = {}

    # Generate a unique key for this image-silo pair
    cache_key = (image_path, silo_path)

    # Check if the bbox is already cached
    if cache_key in bbox_cache:
        print("Loading bbox from cache.")
        return bbox_cache[cache_key]

    # Compute bbox if not cached
    target_regions,im1 = get_objects(image_path)
    if len(target_regions)==0:
      target_regions,im2 = detect_and_draw_bboxes(image_path)
    object_regions,im3 = get_objects(silo_path)
    if len(object_regions)==0:
      object_regions,im4 = detect_and_draw_bboxes(silo_path)
    target_regions = [r for r in target_regions if r[2]-r[0]>100 and r[3]-r[1]>100]
    object_regions = [r for r in object_regions if r[2]-r[0]>100 and r[3]-r[1]>100]
    bbox = find_closest_object(image_path, target_regions, silo_path, object_regions)
    im = add_bbox_to_image(image_path, [bbox])
    cv2_imshow(im)
    # cv2_imshow(im1)
    # cv2_imshow(im2)

    # Save the bbox to cache
    print(bbox)
    bbox_cache[cache_key] = bbox
    with open(pickle_file, 'wb') as f:
        pickle.dump(bbox_cache, f)

    return bbox

# bbox = get_image_bbox("/content/test5/AEDE2320/silo.png","/content/test5/AEDE2320/1.png")
# print(bbox)
# get_objects("/content/tes2/7/silo.png")
# image_path = "/content/test5/AEDE2320/1.png"
# silo_path = "/content/test5/AEDE2320/silo.png"
# target_regions,im = get_objects(image_path)
# target_regions,im = detect_and_draw_bboxes(image_path)
# object_regions,im = get_objects(silo_path)
# object_regions,im = detect_and_draw_bboxes(silo_path)

# cv2_imshow(im)

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]


  0%|          | 0.00/131M [00:00<?, ?B/s]
  3%|▎         | 3.62M/131M [00:00<00:03, 35.5MB/s]
  8%|▊         | 10.1M/131M [00:00<00:02, 54.1MB/s]
 12%|█▏        | 15.4M/131M [00:00<00:02, 53.8MB/s]
 20%|██        | 26.2M/131M [00:00<00:01, 77.1MB/s]
 29%|██▊       | 37.4M/131M [00:00<00:01, 90.7MB/s]
 36%|███▋      | 47.5M/131M [00:00<00:00, 93.3MB/s]
 44%|████▍     | 57.5M/131M [00:00<00:00, 96.8MB/s]
 52%|█████▏    | 67.5M/131M [00:00<00:00, 98.1MB/s]
 59%|█████▉    | 76.9M/131M [00:00<00:00, 95.6MB/s]
 66%|██████▌   | 86.1M/131M [00:01<00:00, 77.1MB/s]
 72%|███████▏  | 94.1M/131M [00:01<00:00, 70.3MB/s]
 78%|███████▊  | 101M/131M [00:01<00:00, 66.9MB/s] 
 84%|████████▍ | 110M/131M [00:01<00:00, 71.2MB/s]
 91%|█████████ | 119M/131M [00:01<00:00, 77.9MB/s]
100%|██████████| 131M/131M [00:01<00:00, 78.9MB/s]


In [ ]:
# image_path = "/content/test5/HPL10632/2.png"
# silo_path = "/content/test5/HPL10632/silo.png"
# get_image_bbox(resize_image(image_path),resize_image(silo_path))
# target_regions,im = get_objects(resize_image(image_path))
# print(target_regions)
# im = add_bbox_to_image(resize_image(image_path), target_regions)
# cv2_imshow(im)
# target_regions,im = detect_and_draw_bboxes(resize_image(image_path))HPL10632
# object_regions,im = get_objects(resize_image(silo_path))
# im = add_bbox_to_image(resize_image(silo_path), object_regions)
# cv2_imshow(im)
# bbox = find_closest_object(resize_image(image_path), target_regions, resize_image(silo_path), object_regions)

# object_regions,im = detect_and_draw_bboxes(resize_image(silo_path))
# bbox = find_closest_object(resize_image(image_path), target_regions, silo_path, object_regions)
# im = add_bbox_to_image(resize_image(image_path), [bbox])
# cv2_imshow(im)

# cv2_imshow(im)

In [ ]:


def analyze_neutrality_and_saturation(image_path,bboxes):
    # image = cv2.imread(image_path)
    # if image is None:
    #     raise FileNotFoundError(f"Image at {image_path} not found.")

    # lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    # hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # # Split LAB channels
    # _, A, B = cv2.split(lab_image)

    # # Define thresholds for neutral colors in A and B channels (LAB)
    # neutral_threshold = 15  # Adjust threshold for desired neutrality sensitivity
    # neutral_mask = (np.abs(A - 128) <= neutral_threshold) & (np.abs(B - 128) <= neutral_threshold)

    # # Calculate the number of neutral pixels
    # neutral_pixels = np.sum(neutral_mask)
    # total_pixels = image.shape[0] * image.shape[1]
    # neutrality_score= (neutral_pixels / total_pixels) * 10

    # # Calculate neutrality score (0 to 100)
    # neutrality_score = round(neutrality_score, 2)

    # # Split HSV channels
    # _, S, _ = cv2.split(hsv_image)

    # # Calculate average saturation level
    # # Ignore fully unsaturated pixels (close to grayscale) to get a meaningful saturation score
    # min_saturation_threshold = 10
    # saturated_pixels = S[S > min_saturation_threshold]
    # if len(saturated_pixels) > 0:
    #     average_saturation = np.mean(saturated_pixels)
    # else:
    #     average_saturation = 0

    # # Normalize saturation score to a range of 0-100
    # saturation_score = round((average_saturation / 255) * 10, 2)

    # result = {
    #     "neutrality_score": neutrality_score,
    #     "saturation_score": saturation_score
    # }
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image at {image_path} not found.")

    # Create an initial mask that includes the entire image
    mask = np.ones(image.shape[:2], dtype=np.uint8) * 255  # White mask (all included)

    # Set the regions inside bounding boxes to 0 (excluded area)
    for bbox in bboxes:
        x, y, w, h = bbox
        mask[y:y+h, x:x+w] = 0  # Exclude the bbox area by setting it to black on the mask

    # Convert the image to LAB and HSV color spaces
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Split LAB channels
    _, A, B = cv2.split(lab_image)

    # Define thresholds for neutral colors in A and B channels (LAB)
    neutral_threshold = 15
    neutral_mask = (np.abs(A - 128) <= neutral_threshold) & (np.abs(B - 128) <= neutral_threshold)

    # Apply the inverted mask to include only areas outside the bounding boxes
    neutral_mask = neutral_mask & (mask == 255)

    # Calculate the number of neutral pixels outside the bounding boxes
    neutral_pixels = np.sum(neutral_mask)
    total_pixels_outside_bboxes = np.sum(mask == 255)
    neutrality_percentage = (neutral_pixels / total_pixels_outside_bboxes) * 4 if total_pixels_outside_bboxes > 0 else 0

    neutrality_score = round(neutrality_percentage, 2)

    # Split HSV channels
    _, S, _ = cv2.split(hsv_image)

    # Apply the inverted mask to the saturation channel
    masked_saturation = S[mask == 255]

    min_saturation_threshold = 10
    saturated_pixels = masked_saturation[masked_saturation > min_saturation_threshold]
    if len(saturated_pixels) > 0:
        average_saturation = np.mean(saturated_pixels)
    else:
        average_saturation = 0

    saturation_score = round((average_saturation / 255) * 4, 2)

    result = {
        "neutrality_score": neutrality_score,
        "saturation_score": saturation_score
    }

    return result

def analyze_neutrality_and_saturation_with_wood(image_path, bboxes):
    import cv2
    import numpy as np

    # Load the image
    image = cv2.imread(image_path)

    if image is None:
        raise FileNotFoundError(f"Image at {image_path} not found.")

    # Create an initial mask that includes the entire image
    mask = np.ones(image.shape[:2], dtype=np.uint8) * 255  # White mask (all included)

    # Set the regions inside bounding boxes to 0 (excluded area)
    for bbox in bboxes:
        x, y, x1, y1 = bbox
        w = x1 - x
        h = y1 - y
        mask[y:y+h, x:x+w] = 0  # Exclude the bbox area by setting it to black on the mask

    # Convert the image to LAB and HSV color spaces
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Split LAB channels
    L, A, B = cv2.split(lab_image)

    # Define thresholds for neutral colors in A and B channels (LAB)
    neutral_threshold = 15
    neutral_mask = (np.abs(A - 128) <= neutral_threshold) & (np.abs(B - 128) <= neutral_threshold)

    # Define thresholds for wood tones based on extracted LAB values
    wood_lab_mean = [143.23, 132.53, 137.33]  # From analysis
    wood_lab_std = [54.36, 5.39, 8.75]       # From analysis

    wood_l_min, wood_l_max = 80, 200  # Broader range for lightness
    wood_a_min = wood_lab_mean[1] - 2 * wood_lab_std[1]
    wood_a_max = wood_lab_mean[1] + 2 * wood_lab_std[1]
    wood_b_min = wood_lab_mean[2] - 2 * wood_lab_std[2]
    wood_b_max = wood_lab_mean[2] + 2 * wood_lab_std[2]

    wood_mask = (
        (wood_l_min <= L) & (L <= wood_l_max) &
        (wood_a_min <= A) & (A <= wood_a_max) &
        (wood_b_min <= B) & (B <= wood_b_max)
    )

    # Combine neutral and wood tone masks
    combined_mask = (neutral_mask | wood_mask) & (mask == 255)

    # Calculate the number of combined neutral and wood tone pixels
    combined_pixels = np.sum(combined_mask)
    total_pixels_outside_bboxes = np.sum(mask == 255)
    neutrality_percentage = (combined_pixels / total_pixels_outside_bboxes) * 4 if total_pixels_outside_bboxes > 0 else 0

    neutrality_score = round(neutrality_percentage, 2)

    # Split HSV channels
    _, S, _ = cv2.split(hsv_image)

    # Apply the mask to the saturation channel
    masked_saturation = S[mask == 255]

    min_saturation_threshold = 10
    saturated_pixels = masked_saturation[masked_saturation > min_saturation_threshold]
    if len(saturated_pixels) > 0:
        average_saturation = np.mean(saturated_pixels)
    else:
        average_saturation = 0

    saturation_score = round((average_saturation / 255) * 4, 2)

    result = {
        "neutrality_score": neutrality_score,
        "saturation_score": saturation_score
    }

    return result


# Example usage
# image_path = "/content/test5/LXIC1793/1.png"
# results = analyze_neutrality_and_saturation_with_wood(image_path,[[49, 119, 675, 700]])
# print(results)

In [ ]:


def get_wall_only(img_path):
  image = cv2.imread(img_path)
  # cv2_imshow(image)
  outputs = predictor(image)
  v = Visualizer(image[:, :, ::-1], coco_metadata, scale=1.2, instance_mode=ColorMode.IMAGE_BW)
  semantic_result = v.draw_sem_seg(outputs["sem_seg"].argmax(0).to("cpu")).get_image()
  # cv2_imshow(semantic_result)


  wall_class_id = 131
  # for idx, class_name in enumerate(coco_metadata.stuff_classes):
  #     if class_name.lower() == "wall-other-merged":
  #         wall_class_id = idx
  #         break
  semantic_segmentation = outputs["sem_seg"].argmax(0).cpu().numpy()

  wall_mask = semantic_segmentation == wall_class_id

  transparent_image = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)

  transparent_image[..., :3] = image
  transparent_image[..., 3] = np.where(wall_mask, 255, 0)
  return transparent_image

# img_path = "/content/2024_11_06_0ov_Kleki.png"
# masked_transparent_image = get_wall_only(img_path)

# cv2_imshow(masked_transparent_image)


In [ ]:
import cv2
import numpy as np

def create_color_strip(start_color, end_color, width=500, height=50):
    strip = np.zeros((height, width, 3), dtype=np.uint8)

    for i in range(width):
        alpha = i / (width - 1)

        color = (1 - alpha) * np.array(start_color) + alpha * np.array(end_color)

        strip[:, i] = color.astype(np.uint8)
    cv2.imwrite("color_strip.png", strip)
    return strip,"color_strip.png"

start_color = (200, 200, 200)
end_color = (255, 255, 255)

color_strip = create_color_strip(start_color, end_color)


In [ ]:
import cv2

def histogram_similarity_score(comparison_image_path):
    _,benchmark_image_path = create_color_strip(start_color, end_color)
    benchmark_image = cv2.imread(benchmark_image_path)
    comparison_image = cv2.imread(comparison_image_path)

    hsv_benchmark = cv2.cvtColor(benchmark_image, cv2.COLOR_BGR2HSV)
    hsv_comparison = cv2.cvtColor(comparison_image, cv2.COLOR_BGR2HSV)

    hist_benchmark = cv2.calcHist([hsv_benchmark], [0, 1, 2], None, [50, 60, 60], [0, 180, 0, 256, 0, 256])
    hist_comparison = cv2.calcHist([hsv_comparison], [0, 1, 2], None, [50, 60, 60], [0, 180, 0, 256, 0, 256])

    cv2.normalize(hist_benchmark, hist_benchmark)
    cv2.normalize(hist_comparison, hist_comparison)

    correlation = cv2.compareHist(hist_benchmark, hist_comparison, cv2.HISTCMP_CORREL)
    score = round((correlation + 1) * 4, 2)
    return score


# comparison_image_path = "/content/images.jpeg"
# score = histogram_similarity_score(comparison_image_path)
# print(f"Histogram Similarity Score: {score}")

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import numpy as np

def feature_similarity_score(comparison_image_path):

    model = VGG16(weights="imagenet", include_top=False)
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    _,benchmark_image_path = create_color_strip(start_color, end_color)

    def extract_features(img_path):
        img = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        features = model.predict(img_data)
        return features.flatten()

    features_benchmark = extract_features(benchmark_image_path)
    features_comparison = extract_features(comparison_image_path)

    cosine_similarity = np.dot(features_benchmark, features_comparison) / (
        np.linalg.norm(features_benchmark) * np.linalg.norm(features_comparison)
    )

    score = min(4.0,round(cosine_similarity * 4, 2))
    return score

# Example usage
# score = feature_similarity_score("/content/download.png")
# print(f"Feature-Based Similarity Score: {score}")

In [ ]:
from sklearn.cluster import KMeans # import KMeans

def score_wall_color(image_path):
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Resize for faster processing
    resized_image = cv2.resize(image, (100, 100))

    # Reshape image into a 2D array of pixels
    pixels = resized_image.reshape(-1, 3)

    # Perform KMeans clustering to find dominant colors
    kmeans = KMeans(n_clusters=3, random_state=42)
    kmeans.fit(pixels)
    dominant_color = kmeans.cluster_centers_[np.argmax(np.bincount(kmeans.labels_))]

    # Calculate the distance to white (255, 255, 255)
    print(dominant_color)
    distance_to_white = np.linalg.norm(dominant_color - np.array([255, 255, 255]))

    # Normalize the distance to a scale of 0 to 4 (lower distance = higher score)
    max_distance = np.linalg.norm(np.array([255, 255, 255]) - np.array([0, 0, 0]))
    score = 4 * (1 - (distance_to_white / max_distance))
    score = max(0, min(4, score))  # Clamp the score between 0 and 4

    return np.round(score, 2)

In [ ]:
from PIL import Image
import numpy as np
def score_wall(image_path):
  image = Image.open(image_path).convert("RGB")

  # Convert the image to a numpy array
  image_data = np.array(image)

  # Define the color thresholds
  lower_bound = np.array([200, 200, 200])
  upper_bound = np.array([255, 255, 255])

  # Create a mask to check which pixels are within the bounds
  mask = np.all((image_data >= lower_bound) & (image_data <= upper_bound), axis=-1)

  # Calculate the percentage of pixels that are close to white
  total_pixels = image_data.shape[0] * image_data.shape[1]
  white_like_pixels = np.sum(mask)
  white_percentage = (white_like_pixels / total_pixels) * 100

  # Calculate a weighted score (0-10) based on closeness to white
  score = (white_percentage / 100) * 4

  white_percentage, score
  return np.round(score, 2)

In [ ]:
def get_wall_color_score(image_path):
  wall_img = get_wall_only(image_path)
  wall_img_path = "/content/wall_img.png"
  cv2.imwrite(wall_img_path, wall_img)
  score3 = score_wall_color(image_path)
  score4 = score_wall(image_path)
  score1 = histogram_similarity_score(wall_img_path)
  score2 = feature_similarity_score(wall_img_path)
  print("score1,score2,score3,score4",score1,score2,score3,score4)
  return np.round((score1*0.2+score2*0.1+score3*0.35+score4),2)

In [ ]:
print(get_wall_color_score("/content/LXIC1793/LXIC1793/2.png"))

[     74.093       72.07      68.481]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 418ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
4.05 0.23 1.12
1.62


In [ ]:
for sku in all_imgs:
  silo_image = [path for path in all_imgs[sku] if path.split("/")[-1].split(".")[0].endswith("silo")]
  if silo_image:
    silo_image = silo_image[0]
  else:
    continue
  for img_path in all_imgs[sku]:
    if img_path.split("/")[-1].split(".")[0].endswith("silo"):
      continue
    print(img_path)
    print(get_wall_color_score(img_path))

/content/test5/ABOM1374/2.png
[      171.6      164.71      155.64]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 616ms/step
5.7 0.21 2.57 0.95
2.96
/content/test5/ABOM1374/3.png
[     226.72      225.64      222.63]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 773ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 603ms/step
6.02 0.24 3.53 1.85
3.7
/content/test5/ABOM1374/1.png
[     213.67      210.96      206.67]


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 990ms/step
4.2 0.24 3.3 1.51
3.17
/content/test5/10/1.png
[     183.69      167.85      153.72]


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 911ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 611ms/step
4.17 0.17 2.63 0.48
2.69
/content/test5/11/1.png
[     230.54      222.35      211.12]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 988ms/step
4.38 0.12 3.46 1.58
3.31
/content/test5/AEDE2320/2.png


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from collections import Counter
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

def load_image(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image_rgb

def approximate_wall_area(image, lower_bound=(0, 0, 0), upper_bound=(255, 255, 255)):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    mask = cv2.inRange(hsv_image, np.array(lower_bound), np.array(upper_bound))

    wall_area = cv2.bitwise_and(image, image, mask=mask)
    return wall_area

def extract_dominant_colors(image, num_colors=3):
    flat_image = image.reshape(-1, 3)

    flat_image = flat_image[np.any(flat_image != [0, 0, 0], axis=1)]

    kmeans = KMeans(n_clusters=num_colors)
    kmeans.fit(flat_image)
    colors = kmeans.cluster_centers_

    label_counts = Counter(kmeans.labels_)
    total_count = sum(label_counts.values())

    color_proportions = {tuple(map(int, colors[i])): count / total_count for i, count in label_counts.items()}
    return color_proportions

def plot_colors(color_proportions):
    sorted_colors = dict(sorted(color_proportions.items(), key=lambda item: item[1], reverse=True))

    plt.figure(figsize=(8, 2))
    plt.axis("off")

    bar = np.zeros((50, 300, 3), dtype="uint8")
    start_x = 0

    for color, proportion in sorted_colors.items():
        end_x = start_x + int(proportion * 300)
        bar[:, start_x:end_x] = color
        start_x = end_x

    plt.imshow(bar)
    plt.show()

    for color, proportion in sorted_colors.items():
        color = "{0}:{1}:{2}".format(color[0], color[1], color[2])
        print(f"Color {color}: {proportion:.2%}")

def analyze_wall_colors(image_path, num_colors=3):
    image = load_image(image_path)
    wall_area = approximate_wall_area(image)

    color_proportions = extract_dominant_colors(wall_area, num_colors)

    plot_colors(color_proportions)

# analyze_wall_colors("/content/masked_transparent_image.png", num_colors=7)

In [ ]:
import os
import zipfile

def unzip_file(zip_file_path, extract_to_path="."):
  """Unzips a zip file to the specified directory.

  Args:
      zip_file_path: The path to the zip file.
      extract_to_path: The directory to extract the files to (defaults to current directory).
  """

  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

  print(f"File '{zip_file_path}' unzipped to '{extract_to_path}'")

# Example usage:
zip_file_path = "/content/drive/MyDrive/100img.zip" # Replace with your zip file path
extract_to_path = "/content/test5" # Replace with your desired extraction directory
unzip_file(zip_file_path, extract_to_path)

File '/content/drive/MyDrive/100img.zip' unzipped to '/content/test5'


In [ ]:
unzip_file("/content/LXIC1793.zip", "/content/LXIC1793")

File '/content/LXIC1793.zip' unzipped to '/content/LXIC1793'


In [ ]:
unzip_file("/content/9.zip", "/content/9")

File '/content/9.zip' unzipped to '/content/9'


In [ ]:
import os
import glob

def get_all_image_paths(main_folder):
    image_paths = {}

    for root, dirs, files in os.walk(main_folder):
        current_folder_images = []

        for ext in ('*.png', '*.jpeg', '*.jpg'):
            current_folder_images.extend(glob.glob(os.path.join(root, ext)))

        if current_folder_images:
            folder_name = os.path.relpath(root, main_folder)
            image_paths[folder_name] = current_folder_images

    return image_paths
all_imgs = get_all_image_paths(extract_to_path)

In [ ]:
all_imgs = get_all_image_paths("/content/LXIC1793")

In [ ]:
all_imgs

{'100img': ['/content/test5/100img/1731052415522_CBOX6655_02.png',
  '/content/test5/100img/1730962587619_CBUV2549_02.png',
  '/content/test5/100img/1730979099908_CBOX1597_02.png',
  '/content/test5/100img/1730888057076_CBXU4571-1.png',
  '/content/test5/100img/1731053492827_CBUP3424_01.png',
  '/content/test5/100img/1730889893922_CBOO5742_02.png',
  '/content/test5/100img/1730891431057_CBOW9334_01.png',
  '/content/test5/100img/1730976869235_FHNF5202_02.png',
  '/content/test5/100img/1730956934541_ESUR9469_02.png',
  '/content/test5/100img/1730881336090_CBUO9583_1.png',
  '/content/test5/100img/1730980989257_CBUV2544_02.png',
  '/content/test5/100img/1730880373571_FHNF8612_01.png',
  '/content/test5/100img/1730966640765_ETRB4172_01.png',
  '/content/test5/100img/1730958880241_CBOO8938_02.png',
  '/content/test5/100img/1730958665684_CBOX3817_01.png',
  '/content/test5/100img/1730954695491_ESUR6086_02.png',
  '/content/test5/100img/1730955953488_CBXU3279_2.png',
  '/content/test5/100img

In [ ]:
# for sku in all_imgs:
#   silo_image = [path for path in all_imgs[sku] if path.split("/")[-1].split(".")[0].endswith("silo")]
#   if silo_image:
#     silo_image = silo_image[0]
#   else:
#     continue
#   for img_path in all_imgs[sku]:
#     if img_path.split("/")[-1].split(".")[0].endswith("silo"):
#       continue
#     print(img_path,silo_image)
#     bbox = get_image_bbox(img_path,silo_image)
#     centered_score, prominence_ratio = centering_prominance(img_path,bbox)
#     results = analyze_neutrality_and_saturation(img_path,[bbox])


#     print(centered_score,prominence_ratio,results)


In [ ]:
def get_bbox_rmbg(input_path):
  import io
  with open(input_path, "rb") as f:
      input_image = f.read()
  output_image = remove(input_image)

  image_without_bg = np.array(Image.open(io.BytesIO(output_image)))
  if image_without_bg.shape[-1] == 4:  # If image has 4 channels (RGBA)
    image_without_bg = cv2.cvtColor(image_without_bg, cv2.COLOR_RGBA2RGB)
  gray_image = cv2.cvtColor(image_without_bg, cv2.COLOR_BGR2GRAY)
  _, thresh = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  bbox = []
  im2 = cv2.imread(input_path)
  for contour in contours:
      x, y, w, h = cv2.boundingRect(contour)
      if h < 30 or w<30:
        continue
      cv2.rectangle(image_without_bg, (x, y), (x + w, y + h), (0, 255, 0), 2)
      bbox.append([x,y,x+w,y+h])
  # im2 = cv2.resize(im2, (1000,1000), interpolation=cv2.INTER_AREA)
  bbox = (min([b[0] for b in bbox]),min([b[1] for b in bbox]),max([b[2] for b in bbox]),max([b[3] for b in bbox]))
  im2[bbox[1]:bbox[3], bbox[0]:bbox[2]] = [255, 255, 255]
  image_name = input_path.split("/")[-1].split(".")[0]
  output_path = input_path.replace(image_name, image_name+"_hero_removed")
  cv2.imwrite(output_path, im2)
  return bbox

In [ ]:
parent_dir = os.path.join("/content/test5/AEDE2320/1.png", os.pardir+"/"+os.pardir)
print(os.path.relpath("/content/test5/AEDE2320/1.png", start=parent_dir),parent_dir)

AEDE2320/1.png /content/test5/AEDE2320/1.png/../..


In [ ]:
import openpyxl
from openpyxl.drawing.image import Image as OpenpyxlImage
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment

wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Product Scores"

headers = ["Product/SKU", "Image","Centering Score", "Prominence Score", "Wall Color Range", "Color Palette Analysis Score"]
header_font = Font(bold=True, size=16)  # Larger, bold font for headers
data_font = Font(size=12)  # Font size for data cells

column_widths = [16, 25, 15, 15, 15, 15]  # Adjust as necessary

row_num = 1
for col_num, (header, width) in enumerate(zip(headers, column_widths), start=1):
    cell = ws.cell(row=1, column=col_num, value=header)
    cell.font = header_font  # Apply header font style
    cell.alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)  # Center align headers with wrap
    ws.column_dimensions[get_column_letter(col_num)].width = width  # Set column width

ws.row_dimensions[row_num].height = 40

row_height = 150
for i,sku in enumerate(all_imgs):
  silo_image = [path for path in all_imgs[sku] if path.split("/")[-1].split(".")[0].endswith("silo")]
  if silo_image:
    silo_image = silo_image[0]
    silo_image = resize_image(silo_image)
  else:
    continue
  for j,img_path in enumerate(all_imgs[sku]):
    img_path = resize_image(img_path)
    print("*"*100)
    print(img_path)
    print("*"*100)

    try:
      if img_path.split("/")[-1].split(".")[0].endswith("silo"):
        continue
      bbox = get_image_bbox(img_path,silo_image)
      im = cv2.imread(img_path)
      print(bbox)

      row_num+=1
      centered_score, prominence_ratio = centering_prominance(img_path,bbox)
      results = analyze_neutrality_and_saturation_with_wood(img_path,[bbox])
      score = get_wall_color_score(img_path)
      ws.cell(row=row_num, column=1, value=sku)
      ws.cell(row=row_num, column=3, value=centered_score)
      ws.cell(row=row_num, column=4, value=prominence_ratio)
      ws.cell(row=row_num, column=5, value=score)
      ws.cell(row=row_num, column=6, value=results["neutrality_score"])
      for col in range(1, 7):
          cell = ws.cell(row=row_num, column=col)
          cell.font = data_font  # Apply font size
          cell.alignment = Alignment(horizontal="center", vertical="center")  # Center align data
      ws.row_dimensions[row_num].height = row_height

      if img_path:
          try:
              img = OpenpyxlImage(img_path)
              img.height = 200
              img.width = 200
              img.anchor = f"{get_column_letter(2)}{row_num}"  # Column "F" for images
              ws.add_image(img)
              # ws.add_image(img, f"{get_column_letter(2)}{row_num}")
              cell = ws.cell(row=row_num, column=2)
              cell.alignment = Alignment(horizontal="center", vertical="center")
          except Exception as e:
              print(f"Error adding image for {i}: {e}")
    except Exception as e:
        import traceback
        print(f"Error adding image for {img_path}: {str(traceback.format_exc())}")
        continue





output_excel_path = "/content/product_scores_with_images.xlsx"
wb.save(output_excel_path)


print(f"Excel file with images saved as {output_excel_path}")

Excel file with images saved as /content/product_scores_with_images.xlsx


In [ ]:
all_imgs

{'100img': ['/content/test5/100img/1731052415522_CBOX6655_02.png',
  '/content/test5/100img/1730962587619_CBUV2549_02.png',
  '/content/test5/100img/1730979099908_CBOX1597_02.png',
  '/content/test5/100img/1730888057076_CBXU4571-1.png',
  '/content/test5/100img/1731053492827_CBUP3424_01.png',
  '/content/test5/100img/1730889893922_CBOO5742_02.png',
  '/content/test5/100img/1730891431057_CBOW9334_01.png',
  '/content/test5/100img/1730976869235_FHNF5202_02.png',
  '/content/test5/100img/1730956934541_ESUR9469_02.png',
  '/content/test5/100img/1730881336090_CBUO9583_1.png',
  '/content/test5/100img/1730980989257_CBUV2544_02.png',
  '/content/test5/100img/1730880373571_FHNF8612_01.png',
  '/content/test5/100img/1730966640765_ETRB4172_01.png',
  '/content/test5/100img/1730958880241_CBOO8938_02.png',
  '/content/test5/100img/1730958665684_CBOX3817_01.png',
  '/content/test5/100img/1730954695491_ESUR6086_02.png',
  '/content/test5/100img/1730955953488_CBXU3279_2.png',
  '/content/test5/100img

In [ ]:
import openpyxl
from openpyxl.drawing.image import Image as OpenpyxlImage
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment

wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Product Scores"

headers = ["Product/SKU", "Image","Centering Score", "Prominence Score", "Wall Color Range", "Color Palette Analysis Score"]
header_font = Font(bold=True, size=16)  # Larger, bold font for headers
data_font = Font(size=12)  # Font size for data cells

column_widths = [16, 25, 15, 15, 15, 15]  # Adjust as necessary

row_num = 1
for col_num, (header, width) in enumerate(zip(headers, column_widths), start=1):
    cell = ws.cell(row=1, column=col_num, value=header)
    cell.font = header_font  # Apply header font style
    cell.alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)  # Center align headers with wrap
    ws.column_dimensions[get_column_letter(col_num)].width = width  # Set column width

ws.row_dimensions[row_num].height = 40

row_height = 150
for sku in all_imgs:
  # silo_image = [path for path in all_imgs[sku] if path.split("/")[-1].split(".")[0].endswith("silo")]
  # if silo_image:
  #   silo_image = silo_image[0]
  #   silo_image = resize_image(silo_image)
  # else:
  #   continue
  for j,img_path in enumerate(all_imgs[sku]):
    img_path = resize_image(img_path)
    print("*"*100)
    print(img_path)
    print("*"*100)

    try:
      if img_path.split("/")[-1].split(".")[0].endswith("silo"):
        continue
      bbox = get_bbox_rmbg(img_path)
      im = cv2.imread(img_path)
      print(bbox)

      row_num+=1
      centered_score, prominence_ratio = centering_prominance(img_path,bbox)
      results = analyze_neutrality_and_saturation_with_wood(img_path,[bbox])
      score = get_wall_color_score(img_path)
      ws.cell(row=row_num, column=1, value=sku)
      ws.cell(row=row_num, column=3, value=centered_score)
      ws.cell(row=row_num, column=4, value=prominence_ratio)
      ws.cell(row=row_num, column=5, value=score)
      ws.cell(row=row_num, column=6, value=results["neutrality_score"])
      for col in range(1, 7):
          cell = ws.cell(row=row_num, column=col)
          cell.font = data_font  # Apply font size
          cell.alignment = Alignment(horizontal="center", vertical="center")  # Center align data
      ws.row_dimensions[row_num].height = row_height

      if img_path:
          try:
              img = OpenpyxlImage(img_path)
              img.height = 200
              img.width = 200
              img.anchor = f"{get_column_letter(2)}{row_num}"  # Column "F" for images
              ws.add_image(img)
              # ws.add_image(img, f"{get_column_letter(2)}{row_num}")
              cell = ws.cell(row=row_num, column=2)
              cell.alignment = Alignment(horizontal="center", vertical="center")
          except Exception as e:
              print(f"Error adding image for {i}: {e}")
    except Exception as e:
        import traceback
        print(f"Error adding image for {img_path}: {str(traceback.format_exc())}")
        continue





output_excel_path = "/content/product_scores_with_images.xlsx"
wb.save(output_excel_path)


print(f"Excel file with images saved as {output_excel_path}")

In [ ]:
import pandas as pd
df = pd.read_excel(output_excel_path)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

print(df)

  Product/SKU  Image  Centering Score  Prominence Score  Wall Color Range  Color Palette Analysis Score
0    LXIC1793    NaN              3.9                 4              1.63                          1.55
1    LXIC1793    NaN              3.9                 4              1.69                          1.98


In [ ]:
# import cv2
# import numpy as np

# def analyze_neutrality_and_saturation(image_path,bbox):
#     image = cv2.imread(image_path)
#     if image is None:
#         raise FileNotFoundError(f"Image at {image_path} not found.")

#     lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
#     hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

#     # Split LAB channels
#     _, A, B = cv2.split(lab_image)

#     # Define thresholds for neutral colors in A and B channels (LAB)
#     neutral_threshold = 15  # Adjust threshold for desired neutrality sensitivity
#     neutral_mask = (np.abs(A - 128) <= neutral_threshold) & (np.abs(B - 128) <= neutral_threshold)

#     # Calculate the number of neutral pixels
#     neutral_pixels = np.sum(neutral_mask)
#     total_pixels = image.shape[0] * image.shape[1]
#     neutrality_score= (neutral_pixels / total_pixels) * 10

#     # Calculate neutrality score (0 to 100)
#     neutrality_score = round(neutrality_score, 2)

#     # Split HSV channels
#     _, S, _ = cv2.split(hsv_image)

#     # Calculate average saturation level
#     # Ignore fully unsaturated pixels (close to grayscale) to get a meaningful saturation score
#     min_saturation_threshold = 10
#     saturated_pixels = S[S > min_saturation_threshold]
#     if len(saturated_pixels) > 0:
#         average_saturation = np.mean(saturated_pixels)
#     else:
#         average_saturation = 0

#     # Normalize saturation score to a range of 0-100
#     saturation_score = round((average_saturation / 255) * 10, 2)

#     result = {
#         "neutrality_score": neutrality_score,
#         "saturation_score": saturation_score
#     }

#     return result

# # Example usage
# image_path = "/content/images.jpeg"
# results = analyze_neutrality_and_saturation(image_path)
# print(f"Results:\nNeutrality Score: {results['Neutrality Score (0-100)']}\nSaturation Score: {results['Saturation Score (0-100)']}")

In [ ]:
# import cv2
# import numpy as np

# def analyze_neutrality_and_saturation_excluding_bbox(image_path, bboxes):
#     # Load the image
#     image = cv2.imread(image_path)
#     if image is None:
#         raise FileNotFoundError(f"Image at {image_path} not found.")

#     # Create an initial mask that includes the entire image
#     mask = np.ones(image.shape[:2], dtype=np.uint8) * 255  # White mask (all included)

#     # Set the regions inside bounding boxes to 0 (excluded area)
#     for bbox in bboxes:
#         x, y, w, h = bbox
#         mask[y:y+h, x:x+w] = 0  # Exclude the bbox area by setting it to black on the mask

#     # Convert the image to LAB and HSV color spaces
#     lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
#     hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

#     # Split LAB channels
#     _, A, B = cv2.split(lab_image)

#     # Define thresholds for neutral colors in A and B channels (LAB)
#     neutral_threshold = 15
#     neutral_mask = (np.abs(A - 128) <= neutral_threshold) & (np.abs(B - 128) <= neutral_threshold)

#     # Apply the inverted mask to include only areas outside the bounding boxes
#     neutral_mask = neutral_mask & (mask == 255)

#     # Calculate the number of neutral pixels outside the bounding boxes
#     neutral_pixels = np.sum(neutral_mask)
#     total_pixels_outside_bboxes = np.sum(mask == 255)
#     neutrality_percentage = (neutral_pixels / total_pixels_outside_bboxes) * 100 if total_pixels_outside_bboxes > 0 else 0

#     # Neutrality score (0 to 100)
#     neutrality_score = round(neutrality_percentage, 2)

#     # Split HSV channels
#     _, S, _ = cv2.split(hsv_image)

#     # Apply the inverted mask to the saturation channel
#     masked_saturation = S[mask == 255]

#     # Filter out low-saturation pixels to focus on meaningful saturation
#     min_saturation_threshold = 10
#     saturated_pixels = masked_saturation[masked_saturation > min_saturation_threshold]
#     if len(saturated_pixels) > 0:
#         average_saturation = np.mean(saturated_pixels)
#     else:
#         average_saturation = 0

#     # Normalize saturation score to a range of 0-100
#     saturation_score = round((average_saturation / 255) * 100, 2)

#     # Final output
#     result = {
#         "Neutrality Score (0-100)": neutrality_score,
#         "Saturation Score (0-100)": saturation_score
#     }

#     return result

# # Example usage
# image_path = "/content/n7ehuh12lxa11.jpg"
# bboxes = [(50, 100, 4, 5), (200, 250, 10, 15)]  # Example bounding boxes: (x, y, width, height)
# results = analyze_neutrality_and_saturation_excluding_bbox(image_path, bboxes)
# print(f"Results:\nNeutrality Score: {results['Neutrality Score (0-100)']}\nSaturation Score: {results['Saturation Score (0-100)']}")

In [ ]:
3) Prominance should consider the aspect ratio of the product as well
4) Wall color should be fine-tuned
